## 1. Importar Librerías

Importamos las librerías necesarias del **Agent Framework**:
- `WorkflowBuilder`: Para construir workflows con múltiples agentes
- `AzureAIAgentClient`: Cliente para crear agentes en Azure AI Foundry
- `AzureCliCredential`: Autenticación usando Azure CLI
- `AgentRunUpdateEvent`: Eventos de streaming de agentes
- `WorkflowOutputEvent`: Evento de salida final del workflow

In [1]:
import asyncio
import os
from collections.abc import Awaitable, Callable
from contextlib import AsyncExitStack
from typing import Any
from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

## 2. Cargar Variables de Entorno

Cargamos la configuración desde el archivo `.env`:
- `AZURE_PROJECT_ENDPOINT`: URL del proyecto de Azure AI Foundry
- `MODEL`: Nombre del modelo deployment

In [2]:
load_dotenv()

print(f"✅ Configuración cargada")
print(f"Endpoint: {os.getenv('AZURE_PROJECT_ENDPOINT')}")
print(f"Modelo: {os.getenv('MODEL')}")

✅ Configuración cargada
Endpoint: https://vicapas-2346-resource.services.ai.azure.com/api/projects/vicapas-2346
Modelo: gpt-4o-mini


## 3. Función Helper para Crear Agentes

Esta función helper gestiona los **async context managers** correctamente:

### ¿Por qué usar AsyncExitStack?
- Permite gestionar múltiples context managers async
- Asegura que credenciales y clientes se cierren correctamente
- Devuelve una factory de agentes y una función de cierre

### Retorna:
- `agent`: Factory async para crear agentes con instrucciones personalizadas
- `close`: Función async para cerrar recursos

In [3]:
async def create_azure_ai_agent() -> tuple[Callable[..., Awaitable[Any]], Callable[[], Awaitable[None]]]:
    """Helper method to create an Azure AI agent factory and a close function.

    This makes sure the async context managers are properly handled.
    """
    stack = AsyncExitStack()
    cred = await stack.enter_async_context(AzureCliCredential())

    client = await stack.enter_async_context(AzureAIAgentClient(async_credential=cred,
                                                                project_endpoint=os.getenv("AZURE_PROJECT_ENDPOINT"),
                                                                model_deployment_name=os.getenv("MODEL")))

    async def agent(**kwargs: Any) -> Any:
        return await stack.enter_async_context(client.create_agent(**kwargs))

    async def close() -> None:
        await stack.aclose()

    return agent, close

print("✅ Función create_azure_ai_agent() definida")

✅ Función create_azure_ai_agent() definida


## 4. Crear los Agentes del Workflow

Definimos dos agentes especializados:

### Writer Agent
- **Rol**: Generador de contenido creativo
- **Instrucciones**: Crear y editar contenido basado en feedback

### Reviewer Agent
- **Rol**: Revisor y crítico constructivo
- **Instrucciones**: Proporcionar feedback accionable de forma concisa

In [4]:
async def create_agents():
    """Crea los agentes Writer y Reviewer"""
    agent_factory, close = await create_azure_ai_agent()
    
    # Create a Writer agent that generates content
    writer = await agent_factory(
        name="Writer",
        instructions=(
            "You are an excellent content writer. You create new content and edit contents based on the feedback."
        ),
    )

    # Create a Reviewer agent that provides feedback
    reviewer = await agent_factory(
        name="Reviewer",
        instructions=(
            "You are an excellent content reviewer. "
            "Provide actionable feedback to the writer about the provided content. "
            "Provide the feedback in the most concise manner possible."
        ),
    )
    
    return writer, reviewer, close

print("✅ Función create_agents() definida")

✅ Función create_agents() definida


## 5. Construir el Workflow

Usamos `WorkflowBuilder` para definir el flujo:

### Arquitectura del Workflow:
```
Input → Writer → Reviewer → Output
```

### Pasos:
1. `set_start_executor(writer)`: El Writer es el primer ejecutor
2. `add_edge(writer, reviewer)`: Conecta Writer → Reviewer
3. `build()`: Compila el workflow

In [5]:
async def build_workflow(writer, reviewer):
    """Construye el workflow con Writer y Reviewer"""
    workflow = (WorkflowBuilder()
        .set_start_executor(writer)
        .add_edge(writer, reviewer)
        .build())
    
    return workflow

print("✅ Función build_workflow() definida")

✅ Función build_workflow() definida


## 6. Ejecutar el Workflow con Streaming

### ¿Qué es el streaming de eventos?
El workflow emite eventos en tiempo real mientras los agentes procesan:

### Tipos de eventos:
- **AgentRunUpdateEvent**: Streaming de tokens generados por cada agente
- **WorkflowOutputEvent**: Resultado final del workflow

### Prompt de ejemplo:
"Create a slogan for a new electric SUV that is affordable and fun to drive."

In [6]:
async def main() -> None:
    """Función principal que ejecuta el workflow completo"""
    agent_factory, close = await create_azure_ai_agent()
    
    try:
        # Crear agentes
        writer = await agent_factory(
            name="Writer",
            instructions=(
                "You are an excellent content writer. You create new content and edit contents based on the feedback."
            ),
        )

        reviewer = await agent_factory(
            name="Reviewer",
            instructions=(
                "You are an excellent content reviewer. "
                "Provide actionable feedback to the writer about the provided content. "
                "Provide the feedback in the most concise manner possible."
            ),
        )

        # Construir workflow
        workflow = (WorkflowBuilder()
            .set_start_executor(writer)
            .add_edge(writer, reviewer).build())
        
        last_executor_id: str | None = None

        # Ejecutar con streaming
        print("\n" + "="*80)
        print("EJECUCIÓN DEL WORKFLOW")
        print("="*80)
        print("\nPrompt: Create a slogan for a new electric SUV that is affordable and fun to drive.\n")
        
        events = workflow.run_stream("Create a slogan for a new electric SUV that is affordable and fun to drive.")
        async for event in events:
            if isinstance(event, AgentRunUpdateEvent):
                # Handle streaming updates from agents
                eid = event.executor_id
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    print(f"\n{eid}:", end=" ", flush=True)
                    last_executor_id = eid
                print(event.data, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                print("\n\n" + "="*80)
                print("RESULTADO FINAL")
                print("="*80)
                print(event.data)
    finally:
        await close()

print("✅ Función main() definida")
print("\n⚠️ Ejecuta la siguiente celda para iniciar el workflow")

✅ Función main() definida

⚠️ Ejecuta la siguiente celda para iniciar el workflow


## 7. Ejecutar el Workflow

Esta celda ejecuta el workflow completo.

**Nota**: Verás el streaming en tiempo real de ambos agentes.

In [7]:
# Ejecutar el workflow
await main()


EJECUCIÓN DEL WORKFLOW

Prompt: Create a slogan for a new electric SUV that is affordable and fun to drive.


Writer: "Electrify Your Journey: Affordable Adventure Awaits!"
Writer: "Electrify Your Journey: Affordable Adventure Awaits!"

Reviewer: 

Reviewer: "Drive the"Drive the Future: Affordable Future: Affordable Fun Fun in Every in Every Charge!" Charge!"

[2025-11-20 17:23:57 - c:\DEV\Workshop\IA\python-ai\.venv\Lib\site-packages\agent_framework\_workflows\_runner.py:167 - WARNING] No outgoing edges found for executor Reviewer; dropping messages.


## Análisis del Resultado

### Flujo de ejecución observado:

1. **Writer**: Genera un slogan creativo para el SUV eléctrico
2. **Reviewer**: Analiza el slogan y proporciona feedback constructivo
3. **Output**: El resultado final se emite cuando el workflow completa

### Ventajas de este patrón:

✅ **Especialización**: Cada agente tiene un rol específico

✅ **Streaming**: Ver el progreso en tiempo real

✅ **Colaboración**: Los agentes trabajan juntos en secuencia

✅ **Reutilizable**: Fácil agregar más agentes al workflow

## Conclusión

Este ejemplo demuestra:

### 1. **Workflows Multi-Agente**
- Coordinación de múltiples agentes especializados
- Paso de información entre agentes
- Flujo de trabajo secuencial

### 2. **Azure AI Agent Client**
- Uso de `AzureAIAgentClient` para crear agentes
- Autenticación con Azure CLI
- Gestión de recursos async con `AsyncExitStack`

### 3. **Event Streaming**
- Procesamiento de eventos en tiempo real
- Distinción entre eventos de actualización y salida
- Visualización del progreso por agente

### 4. **WorkflowBuilder**
- API declarativa para definir workflows
- Configuración clara de ejecutores y edges
- Compilación del grafo de ejecución

### Aplicaciones Prácticas:

✅ **Generación de contenido**: Writer + Editor + Reviewer

✅ **Análisis de documentos**: Extractor + Classifier + Summarizer

✅ **Procesamiento de datos**: Validator + Transformer + Enricher

✅ **Soporte al cliente**: Analyzer + Responder + Quality Checker

### Extensiones posibles:

🔹 Agregar más agentes en la cadena

🔹 Implementar loops de feedback

🔹 Agregar condiciones de routing

🔹 Paralelizar agentes independientes